# Linear Iterative Solvers

The aim is to solve the system:
$$
A \textbf{x} = \textbf{b}
$$
where $A \in \mathbb{R}^{m \times n}$ is a linear operator typically represented as a sparse matrix.

Reference:
* Saad, Y. *Iterative Methods for Sparse Linear Systems*, SIAM, 2003.

We are wrapping [TNL solvers](https://mmg-gitlab.fjfi.cvut.cz/doc/tnl/md_Tutorials_Solvers_Linear_tutorial_Linear_solvers.html) on top of torch tensors.

In [1]:
import torch
from torch.utils.cpp_extension import load

In [2]:
!mkdir -p build
noa_location = '../..'

In [3]:
def generate_tridiagonal(n, l, d, u):
    c = torch.tensor([-1,0,1]).repeat(n)
    r = torch.arange(n).repeat_interleave(3)
    cr = c + r
    rows = r[1:-1]
    cols = cr[1:-1]
    vals = torch.tensor([l, d ,u]).repeat(n)[1:-1]
    Ai = torch.stack([rows, cols])
    A = torch.sparse_coo_tensor(Ai, vals, (n,n))
    return A

In [4]:
n = 100000
A = generate_tridiagonal(n, -0.5, 2.5, -1.5)

In [12]:
A_cu = A.cuda()

In [5]:
Acsr = A.to_sparse_csr()

In [13]:
Acsr_cu = A_cu.to_sparse_csr() # Acsr.cuda() is not supported as of torch 1.10.2

CSR format with indices data over `int` is preferred.

In [9]:
def get_csr_data(csr_matrix):
    values =  csr_matrix.values() 
    crow_indices = csr_matrix.crow_indices().int()
    col_indices = csr_matrix.col_indices().int()
    return (values, crow_indices, col_indices)

In [10]:
Acsr_data = get_csr_data(Acsr)

In [14]:
Acsr_data_cu = get_csr_data(Acsr_cu)

In [15]:
x0 = torch.ones(n)

In [16]:
x0_cu = x0.cuda()

In [24]:
b = Acsr @ x0

/tmp/ipykernel_21820/3677813335.py:1: UserWarning: Pytorch is compiled with MKL LP64 and will convert crow_indices to int32. (Triggered internally at  ../aten/src/ATen/native/mkl/SparseCsrLinearAlgebra.cpp:228.)
  b = Acsr @ x0
/tmp/ipykernel_21820/3677813335.py:1: UserWarning: Pytorch is compiled with MKL LP64 and will convert col_indices to int32. (Triggered internally at  ../aten/src/ATen/native/mkl/SparseCsrLinearAlgebra.cpp:232.)
  b = Acsr @ x0


In [23]:
b_cu = Acsr_cu @  x0_cu

## Stationary methods

Those are fixed point methods. Numerically more robust, they might suffer from computational costs due to slow convergence.
